In [1]:
# packages

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler



In [6]:
# read data

# Get the current working directory
cwd = os.getcwd()
data = pd.read_csv(cwd + '/../datasets/train.csv')

# remove spaces in column names
data.columns = data.columns.str.replace(' ', '')

In [7]:
# summary of data types and missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB


In [8]:
# change the values of the columns Gender and Vehicle_Damage to binary values (0, 1)

data = pd.get_dummies(data, columns=['Gender', 'Vehicle_Damage', 'Driving_License', 'Previously_Insured', 'Response'], drop_first=True)

# encode the values of the column Vehicle_Age to numerical values
le = LabelEncoder()
data['Vehicle_Age'] = le.fit_transform(data['Vehicle_Age'])

# drop id and Vintage columns
data = data.drop(['id', 'Vintage'], axis=1)

# transfrom Annual_Premium to log scale
data['Annual_Premium'] = np.log(data['Annual_Premium'])

In [9]:
# summary of data types and missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 10 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Age                             381109 non-null  int64  
 1   Region_Code                     381109 non-null  float64
 2   Vehicle_Age                     381109 non-null  int64  
 3   Annual_Premium                  381109 non-null  float64
 4   Policy_Sales_Channel            381109 non-null  float64
 5   Gender_Male                     381109 non-null  bool   
 6   Vehicle_Damage_Yes              381109 non-null  bool   
 7   Driving_License_1               381109 non-null  bool   
 8   Previously_Insured_1            381109 non-null  bool   
 9   Response_1                      381109 non-null  bool   
dtypes: bool(5), float64(3), int64(2)
memory usage: 16.4 MB


In [10]:
# select the top 4 values of Policy_Sales_Channel whose proportion is greater than 0.05 and replace the rest with 0
data['Policy_Sales_Channel'] = data['Policy_Sales_Channel'].where(
																															data['Policy_Sales_Channel'].map(
																																data['Policy_Sales_Channel'].value_counts(normalize=True)
																															).ge(0.05), 0
                              															)

# select the top 4 values of Region_Code whose proportion is greater than 0.05 and replace the rest with 0
data['Region_Code'] = data['Region_Code'].where(data['Region_Code'].map(data['Region_Code'].value_counts(normalize=True)).ge(0.04), 0)

# convert the columns Policy_Sales_Channel and Region_Code to integer
data['Policy_Sales_Channel'] = data['Policy_Sales_Channel'].astype(int)
data['Region_Code'] = data['Region_Code'].astype(int)

In [11]:
print(data['Policy_Sales_Channel'].value_counts(normalize=True))

print(data['Region_Code'].value_counts(normalize=True))

Policy_Sales_Channel
152    0.353663
26     0.209127
124    0.194157
0      0.185907
160    0.057146
Name: proportion, dtype: float64
Region_Code
0     0.532144
28    0.279225
8     0.088891
46    0.051820
41    0.047921
Name: proportion, dtype: float64


In [12]:
# apply one-hot encoding to the columns Policy_Sales_Channel and Region_Code
data = pd.get_dummies(data, columns=['Policy_Sales_Channel', 'Region_Code'], drop_first=True)

# Normalize Age, Vehicle_Age, Annual_Premium columns

scaler = StandardScaler()

data[['Age', 'Vehicle_Age', 'Annual_Premium']] = scaler.fit_transform(data[['Age', 'Vehicle_Age', 'Annual_Premium']])

In [13]:
# summary of data types and missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 16 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Age                             381109 non-null  float64
 1   Vehicle_Age                     381109 non-null  float64
 2   Annual_Premium                  381109 non-null  float64
 3   Gender_Male                     381109 non-null  bool   
 4   Vehicle_Damage_Yes              381109 non-null  bool   
 5   Driving_License_1               381109 non-null  bool   
 6   Previously_Insured_1            381109 non-null  bool   
 7   Response_1                      381109 non-null  bool   
 8   Policy_Sales_Channel_26         381109 non-null  bool   
 9   Policy_Sales_Channel_124        381109 non-null  bool   
 10  Policy_Sales_Channel_152        381109 non-null  bool   
 11  Policy_Sales_Channel_160        381109 non-null  bool   
 12  Region_Code_8   

In [15]:
# save the data to a numpy file
np.save(cwd + '/../datasets/train_preprocessed.npy', data)